In [ ]:
from functools import partial
import geopyspark as gps
import fiona
import pyproj

from pyspark import SparkContext
from shapely.geometry import MultiPoint, MultiLineString, shape
from shapely.ops import transform
from geonotebook.wrappers import VectorData, TMSRasterData

In [ ]:
# Set up our spark context 
conf = gps.geopyspark_conf(appName="San Fran MVP", master="local[*]") 
sc = SparkContext(conf=conf)

In [ ]:
# Set the map center to be over San Francisco
M.set_center(-122.45, 37.75, 11)

In [ ]:
# Download the needed geojsons

!curl -o /tmp/bars.geojson https://s3.amazonaws.com/geopyspark-demo/MVP_San_Francisco/bars.geojson
!curl -o /tmp/cafes.geojson https://s3.amazonaws.com/geopyspark-demo/MVP_San_Francisco/cafes.geojson 
!curl -o /tmp/transit.geojson https://s3.amazonaws.com/geopyspark-demo/MVP_San_Francisco/transit.geojson 
!curl -o /tmp/roads.geojson https://s3.amazonaws.com/geopyspark-demo/MVP_San_Francisco/roads.geojson 

In [ ]:
# Read in all of the downloaded geojsons as Shapely geometries

with fiona.open("/tmp/bars.geojson") as source:
    bars_crs = source.crs['init']
    bars = MultiPoint([shape(f['geometry']) for f in source])

with fiona.open("/tmp/cafes.geojson") as source:
    cafes_crs = source.crs['init']
    cafes = MultiPoint([shape(f['geometry']) for f in source])
    
with fiona.open("/tmp/transit.geojson") as source:
    transit_crs = source.crs['init']
    transit = MultiPoint([shape(f['geometry']) for f in source]) 
    
with fiona.open("/tmp/roads.geojson") as source:
    roads_crs = source.crs['init']
    roads = [MultiLineString(shape(line['geometry'])) for line in source]

In [ ]:
# Reproject each Shapely geometry to EPSG:3857 so it can be
# displayed on the map

def create_partial_reprojection_func(crs):
    return partial(pyproj.transform,
                   pyproj.Proj(init=crs),
                   pyproj.Proj(init='epsg:3857'))

reprojected_bars = [transform(create_partial_reprojection_func(bars_crs), bar) for bar in bars]
reprojected_cafes = [transform(create_partial_reprojection_func(cafes_crs), cafe) for cafe in cafes]
reprojected_transit = [transform(create_partial_reprojection_func(transit_crs), trans) for trans in transit]
reprojected_roads = [transform(create_partial_reprojection_func(roads_crs), road) for road in roads]

In [ ]:
# Rasterize the road vectors and create the road fricition
# layer.

rasterize_options = gps.RasterizerOptions(includePartial=True, sampleType='PixelIsArea')

road_raster = gps.rasterize(geoms=reprojected_roads,
                            crs="EPSG:3857",
                            zoom=12,
                            fill_value=1,
                            cell_type=gps.CellType.FLOAT32,
                            options=rasterize_options,
                            num_partitions=50)

road_friction = road_raster.reclassify(value_map={1:1},
                                       data_type=int,
                                       replace_nodata_with=10)

In [ ]:
# Create the cost distance layer for bars based on the
# road network. Then pyramid the layer.

bar_layer = gps.cost_distance(friction_layer=road_friction,
                              geometries=reprojected_bars,
                              max_distance=1500000.0)

bar_pyramid = bar_layer.pyramid()

In [ ]:
# Create the cost distance layer for cafes based on the
# road network. Then pyramid the layer.

cafe_layer = gps.cost_distance(friction_layer=road_friction,
                               geometries=reprojected_cafes,
                               max_distance=1500000.0)

cafe_pyramid = cafe_layer.pyramid()

In [ ]:
# Create the cost distance layer for the transit stops
# based on the road network. Then pyramid the layer.

transit_layer = gps.cost_distance(friction_layer=road_friction,
                                  geometries=reprojected_transit,
                                  max_distance=1500000.0)

transit_pyramid = transit_layer.pyramid()

In [ ]:
# Calculate the weighted layer based on our preferences.

weighted_layer = (-1 * bar_pyramid) + (transit_pyramid * 5) + (cafe_pyramid * 1)

In [ ]:
# Calculate the histogram for the weighted layer and
# then create a ColorRamp from the histogram.

weighted_histogram = weighted_layer.get_histogram()
weighted_color_map = gps.ColorMap.build(breaks=weighted_histogram,
                                        colors='viridis')

In [ ]:
# Build the TMS server from the weighted layer with its
# ColorMap

tms = gps.TMS.build(source=weighted_layer,
                    display=weighted_color_map)

In [ ]:
# Adds the weighted layer and all of the geometries to the map
# Bars are red
# Cafes are orange
# Transit stops are green

M.add_layer(TMSRasterData(tms), name="Weighted Layer")
M.add_layer(VectorData("/tmp/bars.geojson"), name="Bars", colors=[0xff0000])
M.add_layer(VectorData("/tmp/cafes.geojson"), name="Cafes")
M.add_layer(VectorData("/tmp/transit.geojson"), name="Transit", colors=[0x00FF00])

In [ ]:
for l in M.layers:
    M.remove_layer(l)